In [1]:
import os
from glob import glob
from datetime import datetime

import numpy as np

import tensorflow as tf
from tensorflow.keras import layers

from tensorflow.keras import datasets
import matplotlib.pyplot as plt
%matplotlib inline

### hyperparameter tunning

In [2]:
num_epochs = 20
batch_size = 32

learning_rate = 0.001

dropout_rate = 0.5

input_shape = (32,32,3)
num_classes = 10

In [3]:
inputs = layers.Input(input_shape)

net = layers.Conv2D(32,(3,3), padding = 'SAME')(inputs)
net = layers.Activation('relu')(net)
net = layers.Conv2D(32,(3,3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Conv2D(64,(3,3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.Conv2D(64,(3,3), padding = 'SAME')(net)
net = layers.Activation('relu')(net)
net = layers.MaxPooling2D(pool_size=(2,2))(net)
net = layers.Dropout(dropout_rate)(net)

net = layers.Flatten()(net)
net = layers.Dense(512)(net)
net = layers.Activation('relu')(net)
net = layers.Dropout(dropout_rate)(net)
net = layers.Dense(num_classes)(net)
net = layers.Activation('softmax')(net)

model = tf.keras.Model(inputs= inputs, outputs=net, name = 'Basic_CNN')

In [4]:
model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate),
            loss = 'sparse_categorical_crossentropy',
            metrics = ['accuracy'])

### data preprocess

In [5]:
train_paths = glob('cifar/train/*.png')[:1000]
test_paths = glob('cifar/test/*.png')[:1000]

In [6]:
def get_class_name(path):
    return path.split('_')[-1].replace('.png','')

In [7]:
train_labels = [get_class_name(path) for path in train_paths]
class_names = np.unique(train_labels)

In [8]:
def get_label(path):
    fname = tf.strings.split(path, '_')[-1]
    lbl_name = tf.strings.regex_replace(fname, '.png','')
    onehot = tf.cast(lbl_name == class_names, tf.uint8)
    return tf.argmax(onehot) # label번호를 반환

In [9]:
def load_image_label(path):
    gfile = tf.io.read_file(path)
    image = tf.io.decode_image(gfile)
    image = tf.cast(image, tf.float32) / 255.
    
    label = get_label(path)
    return image, label

In [10]:
def image_preprocess(image, label):
    image = tf.image.random_flip_up_down(image)
    image = tf.image.random_flip_left_right(image)
    return image, label

In [11]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

In [12]:
train_dataset = tf.data.Dataset.from_tensor_slices(train_paths)
train_dataset = train_dataset.map(load_image_label, num_parallel_calls = AUTOTUNE)
train_dataset = train_dataset.map(image_preprocess, num_parallel_calls = AUTOTUNE)
train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size = len(train_paths))
train_dataset = train_dataset.repeat()

In [13]:
test_dataset = tf.data.Dataset.from_tensor_slices(test_paths)
test_dataset = test_dataset.map(load_image_label, num_parallel_calls = AUTOTUNE)
test_dataset = test_dataset.batch(batch_size)
test_dataset = test_dataset.repeat()

### check point

In [14]:
save_path = 'checkpoints'

In [15]:
check_point = tf.keras.callbacks.ModelCheckpoint(save_path, monitor='val_acc', verbose = 1)

In [16]:
steps_per_epoch = len(train_paths) //batch_size
validation_steps = len(test_paths) // batch_size

model.fit_generator(
    train_dataset,
    steps_per_epoch = steps_per_epoch,
    validation_data = test_dataset,
    validation_steps = validation_steps,
    epochs = num_epochs,
callbacks = [check_point])

Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
31/31 [==============================] - ETA: 0s - loss: 2.3070 - accuracy: 0.1229
Epoch 00001: saving model to checkpoints
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: checkpoints\assets
31/31 [==============================] - 6s 182ms/step - loss: 2.3070 - accuracy: 0.1229 - val_loss: 2.2930 - val_accuracy: 0.1462
Epoch 2/20
31/31 [==============================] - ETA: 0s - loss: 2.2910 - accuracy: 0.1209
Epoch 00002: saving model to checkpoints
INFO:tensorflow:Assets written to: checkpoints\assets
31/31 [==============================] - 6s 184ms/step - loss: 2.2910 - accuracy: 0.1209 - val_loss: 2.2997 - val_accuracy: 0.0897
Epoch 3/20
31/31 [==============================] 